# Model inferences

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.simplefilter('ignore')

import gc

from os import path
import sys
sys.path.append(path.abspath('..'))

In [ ]:
import os
import cv2
import torch
import pandas as pd
from PIL import Image
from pprint import pprint
import albumentations as albu
from albumentations.pytorch import ToTensorV2
import numpy as np
import onnxruntime as ort

from src.lightning_module import PlanetsModule

In [ ]:
DEVICE = 'cpu'
BATCH_SIZE = 1
DATA_FOLDER = '../dataset/planet/planet/'
ONNX_MODEL_NAME = '../onnx_planet_model.onnx'
!ls {DATA_FOLDER}

In [ ]:
# Загружаем модель
checkpoints_folder = '../experiments/experiment2/'
# Берем какой-то чекпоинт из папки. Если нужен конкретный - можно явно указать путь
checkpoint_name = os.path.join(checkpoints_folder, os.listdir(checkpoints_folder)[0]) 
module = PlanetsModule.load_from_checkpoint(checkpoint_name)

module.eval()
module.to(DEVICE);

In [ ]:
# Загружаем данные
df = pd.read_csv(os.path.join(DATA_FOLDER, 'df_train_ohe.csv'))
names = list(df.columns[1:])
names
df.head()

In [ ]:
# препроцессинг данных 
img_height = 224
img_width = 224
preprocess = albu.Compose(
        [
            albu.Resize(height=img_height, width=img_width),
            albu.Normalize(),
            ToTensorV2(),
        ]
)

In [ ]:
idx = 0
image_name = os.path.join(DATA_FOLDER, 'train-jpg', f'{df["Id"][idx]}.jpg')
image = cv2.imread(image_name)[..., ::-1]
Image.fromarray(image)

In [ ]:
procecces_image = preprocess(image=image)['image']
with torch.no_grad():
    scores = torch.sigmoid(module(procecces_image[None].to(DEVICE)))[0].cpu().numpy()
    
pprint({n:s for s, n in zip(scores, names)})

## Torch model with manual data preprocess

In [ ]:
idx = 0
image_name = os.path.join(DATA_FOLDER, 'train-jpg', f'{df["Id"][idx]}.jpg')
image = cv2.imread(image_name)[..., ::-1]
Image.fromarray(image)

In [ ]:
def torch_preprocessing(
    image,
    image_size = (224, 224),
):
    """
    Convert numpy-image array for inference Torch model.
    """
    # resize
    image = cv2.resize(image.copy(), image_size, interpolation=cv2.INTER_LINEAR)

    # normalize
    mean = np.array((0.485, 0.456, 0.406), dtype=np.float32) * 255.0
    std = np.array((0.229, 0.224, 0.225), dtype=np.float32) * 255.0
    denominator = np.reciprocal(std, dtype=np.float32)
    image = image.astype(np.float32)
    image -= mean
    image *= denominator

    # to tensor and transpose
    image = torch.from_numpy(image.transpose((2, 0, 1)))[None]
    return image


In [ ]:
# готовим тензора для прогона
torch_input = torch_preprocessing(image).to(DEVICE)
torch_input = torch.cat([torch_input] * BATCH_SIZE)

In [ ]:
# прогон через торчовую модель
with torch.no_grad():
    torch_output = module(torch_input)

In [ ]:
torch_output

In [ ]:
scores_torch = torch.sigmoid(torch_output)[0].cpu().numpy()
    
pprint({n:s for s, n in zip(scores_torch, names)})

## Upload onnx model and test it

In [ ]:
# Доступные провайдеры, на которых можно выполнять вычисления
print(ort.get_available_providers())

In [ ]:
# Инициализируем сессию

# При инициализации сессии можно указать несколько провайдеров. Это может быть полезно, если хотим запускать код
# на разных машинах. Например на машине с GPU у нас сработает CUDAExecutionProvider,
# а на машине без GPU CPUExecutionProvider
providers = [
    # 'CUDAExecutionProvider',
    'CPUExecutionProvider',
]

ort_session = ort.InferenceSession(
    ONNX_MODEL_NAME,
    providers=providers
)

print(f'{[input_.name for input_ in ort_session.get_inputs()]}')
print(f'{[output_.name for output_ in ort_session.get_outputs()]}')

In [ ]:
def onnx_preprocessing(
    image,
    image_size=(224, 224)
):
    """
    Convert numpy-image to array for inference ONNX Runtime model.
    """

    # resize
    image = cv2.resize(image.copy(), image_size, interpolation=cv2.INTER_LINEAR)

    # normalize
    mean = np.array((0.485, 0.456, 0.406), dtype=np.float32) * 255.0
    std = np.array((0.229, 0.224, 0.225), dtype=np.float32) * 255.0
    denominator = np.reciprocal(std, dtype=np.float32)
    image = image.astype(np.float32)
    image -= mean
    image *= denominator

    # transpose
    image = image.transpose((2, 0, 1))[None]
    return image

In [ ]:
# готовим входной тензор
onnx_input = onnx_preprocessing(image)
onnx_input = np.concatenate([onnx_input] * BATCH_SIZE)

ort_inputs = {ort_session.get_inputs()[0].name: onnx_input}
print(list(ort_inputs.keys()))

In [ ]:
# выполняем инференс ONNX Runtime
ort_outputs = ort_session.run(None, ort_inputs)[0]

In [ ]:
ort_outputs

In [ ]:
scores_onnx = torch.sigmoid(torch.tensor(ort_outputs))[0].cpu().numpy()
    
pprint({n:s for s, n in zip(scores_onnx, names)})